## Instalar librerias

In [1]:
%%capture
!pip install luigi pyyaml netCDF4 h5netcdf scipy boto3 s3fs numpy tqdm

In [2]:
%%capture
!pip install dask[complete] zarr xarray[io]

In [3]:
%%capture
!pip install --upgrade s3fs

In [4]:
%%capture
!pip install "xarray[complete]"==2023.8.0 s3fs --user

In [5]:
%%capture
!pip install --upgrade dask

## Cargar librerias

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import s3fs
import os
import re
from collections import defaultdict
import zarr
from numcodecs import blosc
from glob import glob
import dask
from dask import delayed, compute
from tqdm import tqdm
from dask.distributed import Client
from dask.diagnostics import ProgressBar
from datetime import datetime

## Funciones

In [2]:
def kelvin_a_celsius(temp_k):
    return xr.where(np.isnan(temp_k), temp_k, temp_k - 273.15)

def extraer_fecha_y_tipo(nombre_archivo):
    match = re.search(r'Temperature-Air-2m-(Max|Mean|Min)-24h.*_(\d{8})_', nombre_archivo)
    if match:
        tipo, fecha = match.groups()
        return fecha, tipo
    return '00000000', ''

In [3]:
#dataframe de archivos

def archivos_mes(directorio):
    archivos = [f for f in os.listdir(directorio) if f.endswith('.nc')]
    data = []

    for archivo in archivos:
        fecha, tipo = extraer_fecha_y_tipo(archivo)
        if fecha != '00000000':
            data.append({
                'año': int(fecha[:4]),
                'mes': int(fecha[4:6]),
                'fecha': pd.to_datetime(fecha, format='%Y%m%d'),
                'tipo': tipo,
                'path': os.path.join(directorio, archivo)
            })

    df = pd.DataFrame(data)
    df.sort_values(by=['fecha', 'tipo'], inplace=True)
    df['tipo'] = pd.Categorical(df['tipo'], categories=['Max', 'Mean', 'Min'], ordered=True)
    df.sort_values(by=['fecha', 'tipo'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df

### Función ejecución mes

In [4]:
def consolidate_nc(df_mes, año, mes):
    start_date = pd.Timestamp(f"{año}-{mes:02d}-01")
    end_date = start_date + pd.offsets.MonthEnd(0)
    datasets = xr.open_mfdataset(df_mes['path'].tolist())
    # Convertir de Kelvin a Celsius
    for var_name in datasets.data_vars:
        datasets[var_name] = kelvin_a_celsius(datasets[var_name])
        datasets[var_name].attrs['units'] = 'C'
    return datasets

In [5]:
def append_zarr(año, mes, df_mes, bucket_salida, ruta_salida_s3):
    datasets = consolidate_nc(df_mes, año, mes)
    s3 = s3fs.S3FileSystem(anon=False)
    zarr_store = s3fs.S3Map(root=f's3://{bucket_salida}/{ruta_salida_s3}', s3=s3)

    # Configurar compresión y encoding
    blosc.set_nthreads(8)
    compressor = zarr.Blosc(cname='lz4', clevel=1, shuffle=False)
    encoding = {vname: {'compressor': compressor, 'chunks': (1, 1801, 3600)} for vname in datasets.data_vars}
    with tqdm(total=100, desc=f"Procesando {año}-{mes:02d}") as pbar:
        try:
            datasets.to_zarr(zarr_store, mode='a', append_dim='time', consolidated=True)
        except ValueError as e:
            # Si el archivo no existe, crear un nuevo archivo
            print(f"Error al abrir el archivo en modo append: {e}. Creando archivo nuevo.")
            datasets.to_zarr(zarr_store, mode='w', consolidated=True, encoding=encoding)
        pbar.update(100)
    print(f"Datos del mes {año}-{mes:02d} procesados y guardados en S3")

### Función para ejecutar un rango de fechas

In [6]:
def procesar_rango_meses(fecha_inicio, fecha_fin, directorio, bucket_salida, ruta_salida_s3):
    # Convertir fechas de entrada a objetos datetime
    fecha_inicio = pd.to_datetime(fecha_inicio)
    fecha_fin = pd.to_datetime(fecha_fin)
    
    # Crear dataframe de todos los archivos a procesar
    df = archivos_mes(directorio)
    
    # Listar todos los meses en el rango de fechas
    rango_meses = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='MS')
    
    # Iterar sobre cada mes en el rango de meses a procesar
    for mes_actual in rango_meses:
        año = mes_actual.year
        mes = mes_actual.month

        # Filtrar archivos para el mes
        df_mes = df[(df['año'] == año) & (df['mes'] == mes)]
        append_zarr(año, mes, df_mes, bucket_salida, ruta_salida_s3)
        print(f"Ejecutado mes: {año}/{mes}")

In [10]:
# Versión 2 dataframe en un rango de fechas

def archivos_mes(directorio, fecha_inicio=None, fecha_fin=None):
    archivos = [f for f in os.listdir(directorio) if f.endswith('.nc')]
    data = []

    for archivo in archivos:
        fecha, tipo = extraer_fecha_y_tipo(archivo)
        if fecha != '00000000':
            data.append({
                'año': int(fecha[:4]),
                'mes': int(fecha[4:6]),
                'fecha': pd.to_datetime(fecha, format='%Y%m%d'),
                'tipo': tipo,
                'path': os.path.join(directorio, archivo)
            })

    df = pd.DataFrame(data)
    df.sort_values(by=['fecha', 'tipo'], inplace=True)
    df['tipo'] = pd.Categorical(df['tipo'], categories=['Max', 'Mean', 'Min'], ordered=True)
    df.sort_values(by=['fecha', 'tipo'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    # Filtrar el DataFrame por el rango de fechas si se proporcionan
    if fecha_inicio and fecha_fin:
        df = df[(df['fecha'] >= pd.to_datetime(fecha_inicio)) & (df['fecha'] <= pd.to_datetime(fecha_fin))]

    return df

In [11]:
# Versión 2 dataframe reducido

def procesar_rango_meses(fecha_inicio, fecha_fin, directorio, bucket_salida, ruta_salida_s3):
    # Convertir fechas de entrada a objetos datetime
    fecha_inicio = pd.to_datetime(fecha_inicio)
    fecha_fin = pd.to_datetime(fecha_fin)
    
    # Crear dataframe de todos los archivos a procesar
    df = archivos_mes(directorio, fecha_inicio, fecha_fin)
    
    # Listar todos los meses en el rango de fechas
    rango_meses = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='MS')
    
    # Iterar sobre cada mes en el rango de meses a procesar
    for mes_actual in rango_meses:
        año = mes_actual.year
        mes = mes_actual.month

        # Filtrar archivos para el mes
        df_mes = df[(df['año'] == año) & (df['mes'] == mes)]
        append_zarr(año, mes, df_mes, bucket_salida, ruta_salida_s3)
        print(f"Ejecutado mes: {año}/{mes}")

### Ejecución rango de fechas

In [ ]:
# Rango 2, 5 años
directorio = '/home/ec2-user/SageMaker/datalake/data/agera/temperature/landing'
fecha_inicio = '2000-01-01'
fecha_fin = '2004-12-31'
bucket_salida = 'climate-action-datalake'
ruta_salida_s3 = 'zone=raw/source=agera5-v1-1/variable=temperature.zarr'

#procesar_rango_meses(fecha_inicio, fecha_fin, directorio, bucket_salida, ruta_salida_s3)

In [ ]:
# Rango 3, 5 años
directorio = '/home/ec2-user/SageMaker/datalake/data/agera/temperature/landing'
fecha_inicio = '2005-01-01'
fecha_fin = '2009-12-31'
bucket_salida = 'climate-action-datalake'
ruta_salida_s3 = 'zone=raw/source=agera5-v1-1/variable=temperature.zarr'

procesar_rango_meses(fecha_inicio, fecha_fin, directorio, bucket_salida, ruta_salida_s3)

In [ ]:
# Rango 4, 5 años
directorio = '/home/ec2-user/SageMaker/datalake/data/agera/temperature/landing'
fecha_inicio = '2010-01-01'
fecha_fin = '2014-12-31'
bucket_salida = 'climate-action-datalake'
ruta_salida_s3 = 'zone=raw/source=agera5-v1-1/variable=temperature.zarr'

procesar_rango_meses(fecha_inicio, fecha_fin, directorio, bucket_salida, ruta_salida_s3)

In [ ]:
# Rango 5, 5 años
directorio = '/home/ec2-user/SageMaker/datalake/data/agera/temperature/landing'
fecha_inicio = '2015-01-01'
fecha_fin = '2019-12-31'
bucket_salida = 'climate-action-datalake'
ruta_salida_s3 = 'zone=raw/source=agera5-v1-1/variable=temperature.zarr'

procesar_rango_meses(fecha_inicio, fecha_fin, directorio, bucket_salida, ruta_salida_s3)

In [ ]:
# Rango 6, 4 años y 6 meses
directorio = '/home/ec2-user/SageMaker/datalake/data/agera/temperature/landing'
fecha_inicio = '2020-01-01'
fecha_fin = '2024-07-09'
bucket_salida = 'climate-action-datalake'
ruta_salida_s3 = 'zone=raw/source=agera5-v1-1/variable=temperature.zarr'

procesar_rango_meses(fecha_inicio, fecha_fin, directorio, bucket_salida, ruta_salida_s3)

In [12]:
# Rango 1, 20 años
directorio = '/home/ec2-user/SageMaker/datalake/data/agera/temperature/landing'
fecha_inicio = '1980-01-01'
fecha_fin = '1999-12-31'
bucket_salida = 'climate-action-datalake'
ruta_salida_s3 = 'zone=raw/source=agera5-v1-1/variable=temperature.zarr'

#procesar_rango_meses(fecha_inicio, fecha_fin, directorio, bucket_salida, ruta_salida_s3)

Procesando 1992-01: 100%|██████████| 100/100 [00:21<00:00,  4.56it/s]


Datos del mes 1992-01 procesados y guardados en S3
Ejecutado mes: 1992/1


Procesando 1992-02: 100%|██████████| 100/100 [00:20<00:00,  4.87it/s]


Datos del mes 1992-02 procesados y guardados en S3
Ejecutado mes: 1992/2


Procesando 1992-03: 100%|██████████| 100/100 [00:19<00:00,  5.01it/s]


...

Datos del mes 1999-10 procesados y guardados en S3
Ejecutado mes: 1999/10


Procesando 1999-11: 100%|██████████| 100/100 [00:23<00:00,  4.17it/s]


Datos del mes 1999-11 procesados y guardados en S3
Ejecutado mes: 1999/11


Procesando 1999-12: 100%|██████████| 100/100 [00:24<00:00,  4.02it/s]

Datos del mes 1999-12 procesados y guardados en S3
Ejecutado mes: 1999/12


## Validar zarr

In [19]:
# Conexión a S3
s3 = s3fs.S3FileSystem(anon=False)
# Ubicación del archivo Zarr en S3
bucket_name = 'climate-action-datalake'
zarr_path1 = 'zone=raw/source=agera5-v1-1/variable=temperature.zarr'
s3_url_1 = f's3://{bucket_name}/{zarr_path1}'
# Abrir el archivo Zarr
ds1 = xr.open_zarr(s3fs.S3Map(s3_url_1, s3=s3))
ds1

<xarray.Dataset>
Dimensions:                      (time: 16262, lat: 1801, lon: 3600)
Coordinates:
  * lat                          (lat) float64 90.0 89.9 89.8 ... -89.9 -90.0
  * lon                          (lon) float64 -180.0 -179.9 ... 179.8 179.9
  * time                         (time) datetime64[ns] 1980-01-01 ... 2024-07-09
Data variables:
    Temperature_Air_2m_Max_24h   (time, lat, lon) float32 dask.array<chunksize=(1, 1801, 3600), meta=np.ndarray>
    Temperature_Air_2m_Mean_24h  (time, lat, lon) float32 dask.array<chunksize=(1, 1801, 3600), meta=np.ndarray>
    Temperature_Air_2m_Min_24h   (time, lat, lon) float32 dask.array<chunksize=(1, 1801, 3600), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.7

### Dask

In [ ]:
def procesar_rango_meses(fecha_inicio, fecha_fin, directorio, bucket_salida, ruta_salida_s3):
    # Convertir fechas de entrada a objetos datetime
    fecha_inicio = pd.to_datetime(fecha_inicio)
    fecha_fin = pd.to_datetime(fecha_fin)
    # Crear dataframe de todos los archivos a procesar
    df = archivos_mes(directorio)
    # Listar todos los meses en el rango de fechas
    rango_meses = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='MS')
    # Crear lista de tareas vacia
    tareas = []
    # Iterar sobre cada mes en el rango de meses a procesar
    for mes_actual in rango_meses:
        año = mes_actual.year
        mes = mes_actual.month

        # Filtrar archivos para el mes
        df_mes = df[(df['año'] == año) & (df['mes'] == mes)]
        
        if not df_mes.empty:
            # Procesar mes
            tarea = dask.delayed(procesar_mes)(df_mes, año, mes, bucket_salida, ruta_salida_s3)
            tareas.append(tarea)
        else:
            print(f"No se encontraron archivos para el mes: {mes}")

    # Iniciar un cliente Dask
    client = dask.distributed.Client(memory_limit='4GB', n_workers=4)

    # Ejecutar todas las tareas en paralelo
    with dask.diagnostics.ProgressBar():
        results = dask.compute(*tareas, scheduler='threads')  # Puedes elegir el scheduler apropiado

    # Cerrar el cliente Dask
    client.close()

    return results

## Ejecución

In [ ]:
# Uso de la función
directorio = '/home/ec2-user/SageMaker/datalake/data/agera/temperature/landing'
fecha_inicio = '1980-06-01'
fecha_fin = '1980-07-31'
bucket_salida = 'climate-action-datalake'
ruta_salida_s3 = 'zone=raw/source=agera5-v1-1/variable=temperature.zarr'
#procesar_rango_meses(fecha_inicio, fecha_fin, directorio, bucket_salida, ruta_salida_s3)